In [1]:
from common.data_pipeline.dataset import DatasetCompiler
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm

datasets = DatasetCompiler(datasets=[mmcbnu(included_portion=0.001), fvusm(included_portion=0)])
compiled_datasets = datasets.compile_datasets()

{<SetType.TRAIN: 'train'>: [DatasetObject(path='./datasets/MMCBNU_6000/ROIs/004/L_Middle/09.bmp', name='004/L_Middle/09.bmp', label=None, mask_path=None, metadata={'finger': 'middle', 'hand': 'left', 'is_augmented': False}), DatasetObject(path='./datasets/MMCBNU_6000/ROIs/001/L_Ring/01.bmp', name='001/L_Ring/01.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'left', 'is_augmented': False}), DatasetObject(path='./datasets/MMCBNU_6000/ROIs/013/L_Middle/01.bmp', name='013/L_Middle/01.bmp', label=None, mask_path=None, metadata={'finger': 'middle', 'hand': 'left', 'is_augmented': False}), DatasetObject(path='./datasets/MMCBNU_6000/ROIs/090/R_Fore/07.bmp', name='090/R_Fore/07.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'right', 'is_augmented': False})], <SetType.TEST: 'test'>: [DatasetObject(path='./datasets/MMCBNU_6000/ROIs/070/R_Ring/01.bmp', name='070/R_Ring/01.bmp', label=None, mask_path=None, metadata={'finger': 'ring', 'hand': 'right', 'i

In [4]:
from common.utll.enums import SetType


datasets.files[SetType.VALIDATION]

[DatasetObject(path='./datasets/MMCBNU_6000/ROIs/043/L_Fore/03.bmp', name='043/L_Fore/03.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'left', 'is_augmented': False})]